In [20]:
import pandas as pd
from collections import Counter
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
df = pd.read_csv(r"C:\Users\Suwani Gunasekara\OneDrive\Documents\Sem 7 ML\Mid term\results.csv")
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [22]:
# get the fifa world cup games only
dfFIFA = df.loc[df.tournament == 'FIFA World Cup'].reset_index()

In [23]:
# get the year column
dfFIFA['Year'] = dfFIFA['date'].apply(lambda x: int(str.split(x,"-")[0]))

In [6]:
#dfFIFA = dfFIFA.drop(columns = ['index','date','city','country','neutral'])
dfFIFA = dfFIFA.drop(columns = ['index','date','city','neutral'])
dfFIFA['home_games'] = 1
dfFIFA['away_games'] = 1
dfFIFA.head()

,home_team,away_team,home_score,away_score,tournament,country,Year,home_games,away_games
0,Belgium,United States,0,3,FIFA World Cup,Uruguay,1930,1,1
1,France,Mexico,4,1,FIFA World Cup,Uruguay,1930,1,1
2,Brazil,Yugoslavia,1,2,FIFA World Cup,Uruguay,1930,1,1
3,Peru,Romania,1,3,FIFA World Cup,Uruguay,1930,1,1
4,Argentina,France,1,0,FIFA World Cup,Uruguay,1930,1,1


In [24]:
# get the year and country finals held
dfFIFACountry = dfFIFA[['Year','country']]
dfFIFACountry = dfFIFACountry.drop_duplicates().reset_index().drop(columns = ['index'])
dfFIFACountry

,Year,country
0,1930,Uruguay
1,1934,Italy
2,1938,France
3,1950,Brazil
4,1954,Switzerland
5,1958,Sweden
6,1962,Chile
7,1966,England
8,1970,Mexico
9,1974,Germany


In [8]:
#dfFIFA.loc[(dfFIFA.home_team=='Chile') & (dfFIFA.Year == 1930)]

In [9]:
dfFIFA_home = dfFIFA.groupby(['Year','home_team']).sum().rename(columns = {'away_score':'home_goals_conceded'}).drop(columns ='away_games') #.reset_index().set_index('Year')
dfFIFA_home.head()

home_score  home_goals_conceded  home_games
Year home_team                                             
1930 Argentina          16                    5           4
     Belgium             0                    4           2
     Bolivia             0                    8           2
     Brazil              1                    2           1
     Chile               4                    0           2

In [10]:
dfFIFA_away = dfFIFA.groupby(['Year','away_team']).sum().rename(columns = {'home_score':'away_goals_conceded'}).drop(columns = ['home_games'])#.reset_index().set_index('Year')
dfFIFA_away.head()

away_goals_conceded  away_score  away_games
Year away_team                                             
1930 Argentina                    4           2           1
     Brazil                       0           4           1
     Chile                        3           1           1
     France                       2           0           2
     Mexico                      13           4           3

In [11]:
dfFIFA_away_ri = dfFIFA_away.reset_index()
dfFIFA_away_ri.loc[dfFIFA_away_ri.Year == 1930]

,Year,away_team,away_goals_conceded,away_score,away_games
0,1930,Argentina,4,2,1
1,1930,Brazil,0,4,1
2,1930,Chile,3,1,1
3,1930,France,2,0,2
4,1930,Mexico,13,4,3
5,1930,Paraguay,0,1,1
6,1930,Peru,1,0,1
7,1930,Romania,5,3,2
8,1930,United States,6,7,3
9,1930,Yugoslavia,7,7,3


In [12]:
dfFIFAdata = pd.concat([dfFIFA_away, dfFIFA_home.reindex(dfFIFA_away.index)], axis=1).reset_index().set_index('Year').rename(columns = {'away_team':'team'})
dfFIFAdata = dfFIFAdata.fillna(0)
dfFIFAdata.head()

,team,away_goals_conceded,away_score,away_games,home_score,home_goals_conceded,home_games
Year,,,,,,,
1930,Argentina,4,2,1,16.0,5.0,4.0
1930,Brazil,0,4,1,1.0,2.0,1.0
1930,Chile,3,1,1,4.0,0.0,2.0
1930,France,2,0,2,4.0,1.0,1.0
1930,Mexico,13,4,3,0.0,0.0,0.0


In [13]:
dfFIFA_total_avg_1 = dfFIFA.groupby(['Year']).mean().rename(columns = {'home_score':'hs_mean','away_score':'as_mean'}).drop(columns = ['home_games','away_games'])
dfFIFA_total_avg_1.head()

,hs_mean,as_mean
Year,,
1930,2.277778,1.611111
1934,2.235294,1.882353
1938,2.555556,2.111111
1950,1.954545,2.045455
1954,2.730769,2.653846


In [14]:
dfFIFA_total_avg_2 = dfFIFA_total_avg_1.copy().rename(columns = {'hs_mean':'as_conceded_mean','as_mean':'hs_conceded_mean'})
dfFIFA_total_avg_2.head()

,as_conceded_mean,hs_conceded_mean
Year,,
1930,2.277778,1.611111
1934,2.235294,1.882353
1938,2.555556,2.111111
1950,1.954545,2.045455
1954,2.730769,2.653846


In [15]:
dfFIFA_total_avg = pd.concat([dfFIFA_total_avg_1, dfFIFA_total_avg_2], axis=1, join='inner')
dfFIFA_total_avg.head()

,hs_mean,as_mean,as_conceded_mean,hs_conceded_mean
Year,,,,
1930,2.277778,1.611111,2.277778,1.611111
1934,2.235294,1.882353,2.235294,1.882353
1938,2.555556,2.111111,2.555556,2.111111
1950,1.954545,2.045455,1.954545,2.045455
1954,2.730769,2.653846,2.730769,2.653846


In [16]:
dfFIFAdata = dfFIFAdata.merge(dfFIFA_total_avg, on='Year').reset_index()
dfFIFAdata.head()

,Year,team,away_goals_conceded,away_score,away_games,home_score,home_goals_conceded,home_games,hs_mean,as_mean,as_conceded_mean,hs_conceded_mean
0,1930,Argentina,4,2,1,16.0,5.0,4.0,2.277778,1.611111,2.277778,1.611111
1,1930,Brazil,0,4,1,1.0,2.0,1.0,2.277778,1.611111,2.277778,1.611111
2,1930,Chile,3,1,1,4.0,0.0,2.0,2.277778,1.611111,2.277778,1.611111
3,1930,France,2,0,2,4.0,1.0,1.0,2.277778,1.611111,2.277778,1.611111
4,1930,Mexico,13,4,3,0.0,0.0,0.0,2.277778,1.611111,2.277778,1.611111


In [17]:
dfFIFAdata['H_AttackS'] = (dfFIFAdata['home_score']/dfFIFAdata['home_games'])/dfFIFAdata['hs_mean']
dfFIFAdata['A_AttackS'] = (dfFIFAdata['away_score']/dfFIFAdata['away_games'])/dfFIFAdata['as_mean']
dfFIFAdata['H_DefenseS'] = (dfFIFAdata['home_goals_conceded']/dfFIFAdata['home_games'])/dfFIFAdata['hs_conceded_mean']
dfFIFAdata['A_DefenseS'] = (dfFIFAdata['away_goals_conceded']/dfFIFAdata['away_games'])/dfFIFAdata['as_conceded_mean']

In [18]:
dfFIFAdata = dfFIFAdata.set_index(['Year','team'])
dfFIFAdata.head()

away_goals_conceded  away_score  away_games  home_score  \
Year team                                                                 
1930 Argentina                    4           2           1        16.0   
     Brazil                       0           4           1         1.0   
     Chile                        3           1           1         4.0   
     France                       2           0           2         4.0   
     Mexico                      13           4           3         0.0   

                home_goals_conceded  home_games   hs_mean   as_mean  \
Year team                                                             
1930 Argentina                  5.0         4.0  2.277778  1.611111   
     Brazil                     2.0         1.0  2.277778  1.611111   
     Chile                      0.0         2.0  2.277778  1.611111   
     France                     1.0         1.0  2.277778  1.611111   
     Mexico                     0.0         0.0  2.277778  1.611111   

                as_conceded_mean  hs_conceded_mean  H_AttackS  A_AttackS  \
Year team                                                                  
1930 Argentina          2.277778          1.611111   1.756098   1.241379   
     Brazil             2.277778          1.611111   0.439024   2.482759   
     Chile              2.277778          1.611111   0.878049   0.620690   
     France             2.277778          1.611111   1.756098   0.000000   
     Mexico             2.277778          1.611111        NaN   0.827586   

                H_DefenseS  A_DefenseS  
Year team                               
1930 Argentina    0.775862    1.756098  
     Brazil       1.241379    0.000000  
     Chile        0.000000    1.317073  
     France       0.620690    0.439024  
     Mexico            NaN    1.902439

In [19]:
yearlist = dfFIFA.Year.unique().tolist()
yearlist

[1930,
 1934,
 1938,
 1950,
 1954,
 1958,
 1962,
 1966,
 1970,
 1974,
 1978,
 1982,
 1986,
 1990,
 1994,
 1998,
 2002,
 2006,
 2010,
 2014,
 2018]

In [37]:
contdic={}

for year in yearlist:
    #print(year)
    wonlist = []
    for index,row in dfFIFA.iterrows():
        if row['Year'] == year:
            if row['home_score']>row['away_score']:
                wonlist.append(row['home_team'])
            elif row['home_score']<row['away_score']:
                wonlist.append(row['away_team'])
    contdic[year] = wonlist

In [38]:
k = {}
for year in yearlist:
    k[year] = Counter(contdic[year])

In [39]:
# to make a df with countries and corrosponding number of wins
dflist = []
for year in yearlist:
    df = pd.DataFrame.from_dict(k[year],orient = 'index').reset_index().rename(columns ={'index':'team',0:'WinCount'} )
    df['Year'] = year
    # this is a list of data frames
    dflist.append(df)


In [40]:
result = dflist[0].append(dflist[1:]).set_index(['Year','team'])
result.head()

WinCount
Year team                   
1930 United States         2
     France                1
     Yugoslavia            2
     Romania               1
     Argentina             4

In [41]:
pd.merge(dfFIFAdata, result, left_index=True, right_index=True)

,,away_goals_conceded,away_score,away_games,home_score,home_goals_conceded,home_games,hs_mean,as_mean,as_conceded_mean,hs_conceded_mean,H_AttackS,A_AttackS,H_DefenseS,A_DefenseS,WinCount
Year,team,,,,,,,,,,,,,,,
1986,Germany,6,6,6,2.0,1.0,1.0,1.461538,1.076923,1.461538,1.076923,1.368421,0.928571,0.928571,0.684211,3
1934,Spain,3,4,3,0.0,0.0,0.0,2.235294,1.882353,2.235294,1.882353,NaN,0.708333,NaN,0.447368,1
1982,Russia,2,2,3,5.0,2.0,2.0,1.826923,0.980769,1.826923,0.980769,1.368421,0.679739,1.019608,0.364912,2
1950,Switzerland,3,4,2,0.0,3.0,1.0,1.954545,2.045455,1.954545,2.045455,0.000000,0.977778,1.466667,0.767442,1
1990,Argentina,2,2,4,3.0,2.0,3.0,1.269231,0.942308,1.269231,0.942308,0.787879,0.530612,0.707483,0.393939,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006,Switzerland,0,2,2,2.0,0.0,2.0,1.390625,0.906250,1.390625,0.906250,0.719101,1.103448,0.000000,0.000000,2
1930,United States,6,7,3,0.0,0.0,0.0,2.277778,1.611111,2.277778,1.611111,NaN,1.448276,NaN,0.878049,2
1978,Tunisia,2,3,3,0.0,0.0,0.0,1.473684,1.210526,1.473684,1.210526,NaN,0.826087,NaN,0.452381,1


In [42]:
dfFIFAdata_final = dfFIFAdata.join(result)

In [43]:
dfFIFAdata_final = dfFIFAdata_final.fillna(0).reset_index()
dfFIFAdata_final.head()

,Year,team,away_goals_conceded,away_score,away_games,home_score,home_goals_conceded,home_games,hs_mean,as_mean,as_conceded_mean,hs_conceded_mean,H_AttackS,A_AttackS,H_DefenseS,A_DefenseS,WinCount
0,1930,Argentina,4,2,1,16.0,5.0,4.0,2.277778,1.611111,2.277778,1.611111,1.756098,1.241379,0.775862,1.756098,4.0
1,1930,Brazil,0,4,1,1.0,2.0,1.0,2.277778,1.611111,2.277778,1.611111,0.439024,2.482759,1.241379,0.000000,1.0
2,1930,Chile,3,1,1,4.0,0.0,2.0,2.277778,1.611111,2.277778,1.611111,0.878049,0.620690,0.000000,1.317073,2.0
3,1930,France,2,0,2,4.0,1.0,1.0,2.277778,1.611111,2.277778,1.611111,1.756098,0.000000,0.620690,0.439024,1.0
4,1930,Mexico,13,4,3,0.0,0.0,0.0,2.277778,1.611111,2.277778,1.611111,0.000000,0.827586,0.000000,1.902439,0.0


In [91]:
#dfFIFAdata_final.to_csv(r"C:\Users\Suwani Gunasekara\OneDrive\Documents\Sem 7 ML\Mid term\dfFIFAdata_final.csv")

In [46]:
dfFIFAdata_final = dfFIFAdata_final.merge(dfFIFACountry, on = 'Year')

In [47]:
#dfFIFAdata_final.to_csv(r"C:\Users\Suwani Gunasekara\OneDrive\Documents\Sem 7 ML\Mid term\dfFIFAdata_final_wcountry.csv")